Let's start by downloading a small portion of data. NASA's PDS-PPI node for MESSENGER stores full cadence data in daily files, so we will just download one of these.

In [1]:
import requests
import os

# 1st September 2011
url = "https://pds-ppi.igpp.ucla.edu/data/mess-mag-calibrated/data/mso/2011/244_273_SEP/MAGMSOSCI11244_V08.TAB"

save_dir = "./data/tmp/"
filename = os.path.basename(url)
save_path = os.path.join(save_dir, filename)
os.makedirs(save_dir, exist_ok=True)

with requests.get(url, stream=True) as r:
    r.raise_for_status()
    with open(save_path, "wb") as f:
        for chunk in r.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)

print(f"Data saved to: {save_path}")

Data saved to: ./data/tmp/MAGMSOSCI11244_V08.TAB


We require a SPICE metakernel for MESSENGER. This can be created automatically with the below code

In [ ]:
os.system("chmod +x ../scripts/make_messenger_metakernel")
os.system("../scripts/make_messenger_metakernel")

Cloning mjrutala/AutoMeta to pull SPICE kernels


Cloning into './data/autometa'...


/home/daraghhollman/Main/Work/mercury/messenger_region_classifier/.venv/bin/python3: can't open file '/home/daraghhollman/Main/Work/mercury/messenger_region_classifier/examples/src/make_messenger_metakernel.py': [Errno 2] No such file or directory


512

We need to tell the codebase where to expect our data files:

In [ ]:
import hermpy.utils

hermpy.utils.User.DATA_DIRECTORIES["MAG_FULL"] = "./data/tmp/"
hermpy.utils.User.METAKERNEL = "./SPICE/messenger/metakernel_messenger.txt"